In [42]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

In [43]:
import minsearch

In [44]:
import json 

In [45]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [46]:
#print(docs_raw)

In [47]:
documents= []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']
        documents.append(doc)
    

In [48]:
#documents

In [49]:
index=minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields= ["course"]
)

In [50]:
#q= 'the course has already started, can I still enroll?'

In [51]:
index.fit(documents)

In [52]:
#results

In [53]:
import  ollama

In [54]:
#!ollama pull phi3

In [55]:
#!ollama pull mistral

In [56]:
from openai import OpenAI

In [57]:
client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama'  # required, but unused
)

In [58]:
#print(prompt)

In [59]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results=index.search(
        query = query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost, 
        num_results=5
    )

    return results

In [60]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION. 

QUESTION : {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
         context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt=prompt_template.format(question=query, context=context).strip()
    return prompt

In [61]:
def llm(prompt):
    response = client .chat.completions.create(
           model="phi3",
           messages=[{"role":"user", "content":prompt}]
    )
    
    return response.choices[0].message.content

In [62]:
query= 'how do I run kafka? '




In [28]:
rag(query)

" To run Kafka in the terminal, you need to ensure that your Java project setup for streaming with Kafka is correct. However, based on the provided CONTEXT from Module 6 of our FAQ database, it doesn't directly state how to run a Kafka producer/consumer using the terminal command line but does give instructions to start a Java application involving Kafka:\n\nIn your project directory, you should first set up your Java environment correctly. Assuming 'build/libs/' holds your necessary jar files for running a Kafka-related Java application, you might need something similar to this in your terminal:\n\n```bash\njava -cp build/libs/*-SNAPSHOT.jar:out yourJavaClassName\n```\n\nReplace `yourJavaClassName` with the actual class that includes your producer or consumer logic for Kafka. However, please note that without specific details on the application code and dependencies within 'build/libs/' directory, I can only provide a general example as per the FAQ's format. The exact command will var

In [63]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [83]:
from elasticsearch import Elasticsearch


In [84]:
es_client= Elasticsearch('http://localhost:9200')

In [85]:
index_settings= {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index= index_name , body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [88]:
!pip install tqdm

In [89]:
from tqdm import tqdm


In [90]:
for doc in tqdm(documents) :
    es_client.index(index= index_name , document=doc )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:27<00:00, 34.35it/s]


In [99]:
query= 'I just discovered the course. Can I still join it?'

In [104]:
def elastic_search(query):
    search_query={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response= es_client.search(index=index_name, body=search_query)
    
    result_docs= []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [106]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the 

In [107]:
def rag(query):
    search_results = elastic_search(query)
    prompt=build_prompt(query, search_results)
    answer= llm(prompt)
    return answer

In [108]:
rag(query)

" Yes, even if you don't register before the start date of the course, you can still join by submitting homeworks as there will be deadlines for turning in final projects. However, it is recommended not to leave everything until the last minute.\n\nAdditionally, after the course finishes, we keep all the materials available and allow you to follow the course at your own pace, including working on your final capstone project and looking at homeworks as preparation for the next cohort."